In [1]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install hf_transfer

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.2/218.2 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

https://huggingface.co/google/gemma-7b-it

In [ ]:
!nvidia-smi

Thu Feb 22 11:15:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import os


#os.environ ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'

In [2]:
torch.set_default_device('cuda')

In [3]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it", # 'google/gemma-2b-it
                                             quantization_config=quantization_config,
                                             low_cpu_mem_usage=True,
                                             torch_dtype="auto",
                                             device_map="auto"
                                             )

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

In [4]:
tokenizer.vocab_size

256000

In [5]:
chat = [
    { "role": "user", "content": "What is the difference between LLaMAs, Alpacas, and Vicunas" },
]

prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
prompt


'<start_of_turn>user\nWhat is the difference between LLaMAs, Alpacas, and Vicunas<end_of_turn>\n<start_of_turn>model\n'

In [6]:
inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt")

outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=512)

In [8]:
from IPython.display import Markdown, display
display(Markdown("## Gemma - 7B 4Bit"))

## Gemma - 7B 4Bit

In [9]:
text = tokenizer.decode(outputs[0],skip_special_tokens=True, clean_up_tokenization_spaces=True)
# text = text.replace(prompt, '', 1)
display(Markdown(text))

user
What is the difference between LLaMAs, Alpacas, and Vicunas
model
Sure, here is the difference between LLamas, Alpacas, and Vicunas:

**LLamas:**
- Found in North America and South America
- Domesticated by humans for centuries
- Come in a variety of colors, including black, white, gray, and brown
- Have a thick coat of wool that is soft and warm
- Can be used for a variety of purposes, including transportation, companionship, and fiber production

**Alpacas:**
- Found in South America
- Domesticated by humans for centuries
- Come in a variety of colors, including white, gray, black, and brown
- Have a thick coat of wool that is soft and warm
- Can be used for a variety of purposes, including transportation, companionship, and fiber production

**Vicunas:**
- Found in South America
- Domesticated by humans for centuries
- Come in a variety of colors, including white, gray, black, and brown
- Have a thick coat of wool that is soft and warm
- Can be used for a variety of purposes, including transportation, companionship, and fiber production

**Key differences:**

- **LLamas** are the most common species of llama, while **Alpacas** and **Vicunas** are the two rarest species.
- **Alpacas** and **Vicunas** have a thicker coat of wool than LLamas.
- **Vicunas** have a more dense coat of wool than Alpacas.
- **LLamas** are found in North America and South America, while **Alpacas** and **Vicunas** are found in South America.

In [10]:
chat = [
    { "role": "user", "content": "What is the difference between LLaMAs, Alpacas, and Vicunas" },
]

prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
prompt


'<start_of_turn>user\nWhat is the difference between LLaMAs, Alpacas, and Vicunas<end_of_turn>\n<start_of_turn>model\n'

In [11]:
prompt

'<start_of_turn>user\nWhat is the difference between LLaMAs, Alpacas, and Vicunas<end_of_turn>\n<start_of_turn>model\n'

### Prompt Format
```
<start_of_turn>user
What is the difference between LLaMAs, Alpacas, and Vicunas<end_of_turn>
<start_of_turn>model



```

In [12]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [13]:
def generate(input_text, system_prompt="",max_length=512):
    if system_prompt != "":
        system_prompt = system_prompt
    else:
        system_prompt = "You are a friendly and helpful assistant"
    messages = [
        # {
        #     "role": "system",
        #     "content": system_prompt,
        # },
        {"role": "user", "content": system_prompt + '\n\n' +input_text},
    ]

    prompt = tokenizer.apply_chat_template(messages,
                                                tokenize=False,
                                                add_generation_prompt=True)

    inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt")
    outputs = model.generate(input_ids=inputs.to(model.device),
                             max_new_tokens=max_length,
                             do_sample=True,
                             temperature=0.1,
                             top_k=50,
                             )
    text = tokenizer.decode(outputs[0],skip_special_tokens=True, clean_up_tokenization_spaces=True)
    text = text.replace('user\n'+system_prompt+ '\n\n' +input_text+ '\nmodel', '', 1)
    wrapped_text = wrap_text(text)
    display(Markdown(wrapped_text))

## Instruction Answering

In [14]:
generate('Write a detailed analogy between mathematics and a lighthouse.',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)


**Reasoning:**

**Step 1: Identify the key similarities between mathematics and a lighthouse.**

* **Mathematics:** A complex system of principles and concepts that deal with numbers,
shapes, and other abstract ideas.
* **Lighthouse:** A tall structure that provides a visible point of reference for sailors.

**Step 2: Determine the role of each element in its respective domain.**

* **Mathematics:** The foundation of mathematics lies in its ability to abstract and
quantify abstract concepts, providing a framework for solving problems and making
predictions.
* **Lighthouse:** The lighthouse's primary function is to provide a beacon of light,
guiding sailors towards the shore.

**Step 3: Draw the analogy.**

* The **principles and concepts of mathematics** are like the guiding principles and rules
of navigation.
* The **complex calculations and formulas in mathematics** are like the intricate
calculations and adjustments made by sailors to determine their position.
* The **solutions to mathematical problems** are like the safe harbor that sailors reach
after following the guidance of the lighthouse.

**Conclusion:**

The analogy between mathematics and a lighthouse highlights the similarities between the
two domains. Mathematics provides a framework for abstract thought and problem-solving,
while the lighthouse offers a visible guide for sailors. Just as the lighthouse is an
essential beacon for sailors, mathematics is an essential tool for understanding and
navigating the complexities of the world.

In [15]:
%%time
generate('Write a detailed analogy between mathematics and a music.',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=256)


**Reasoning:**

**Step 1: Identify the similarities between mathematics and music.**

* **Structure:** Mathematics and music both have a structured framework. In mathematics,
there are rules for arithmetic, algebra, and geometry. In music, there are rules for
pitch, rhythm, and harmony.
* **Elements:** Both disciplines have elements that are essential to their respective
fields. In mathematics, these elements include numbers, variables, and operators. In
music, they include notes, chords, and rhythms.
* **Expression:** Both mathematics and music are expressive mediums. In mathematics,
formulas and equations can be used to express complex ideas. In music, composers use notes
and rhythms to express emotions and stories.

**Step 2: Identify the differences between mathematics and music.**

* **Objective vs. subjective:** Mathematics is an objective discipline, while music is
subjective. In mathematics, there is a right answer for every problem. In music, there is
no right or wrong answer, as it is subjective to the listener's ears.
* **Linear vs. cyclical:** Mathematics is a linear discipline, while music is cyclical. In
mathematics, concepts are presented in a sequential order. In music, concepts are repeated
and revisited throughout the piece.
* **Abstract

CPU times: user 27.8 s, sys: 123 ms, total: 27.9 s
Wall time: 28 s


In [16]:
%%time
generate('What is the difference between a Llama, Vicuna and an Alpaca?',
         system_prompt="Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)


**Reasoning:**

**1. Breed Classification:**
- Llama, Vicuna, and Alpaca are all breeds of Camelids, which are herbivores that live in
North America, South America, and the Middle East.

**2. Coat Characteristics:**
- Llama has thick, long hair that can grow up to 12 inches.
- Vicuna has exceptionally fine, soft hair that is highly prized for its warmth and
softness.
- Alpaca has dense, lustrous fleece that is known for its softness and warmth.

**3. Color Variations:**
- Llama comes in various colors, including white, gray, black, and brown.
- Vicuna has a wide range of color variations, including fawn, gray, black, and white.
- Alpaca has a distinctive range of fleece colors, including white, fawn, gray, black, and
rose.

**4. Temperament:**
- Llama is known for its gentle and calm temperament.
- Vicuna is generally friendly and easy to handle.
- Alpaca is known for its intelligence and intelligence.

**5. Uses:**
- Llama is often used for meat production, fiber, and companionship.
- Vicuna is prized for its luxurious fiber, which is used in high-end textiles.
- Alpaca is primarily raised for fiber production, companionship, and show purposes.

**Conclusion:**

The primary difference between a Llama, Vicuna, and Alpaca is their coat characteristics,
breed classification, color variations, temperament, and uses. Llama has thick hair,
Vicuna has exceptionally fine hair, and Alpaca has dense fleece. They are all breeds of
Camelids, sharing similar physical traits and behaviors.

CPU times: user 39.6 s, sys: 184 ms, total: 39.8 s
Wall time: 42 s


In [17]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)


**Subject: Reasons to Open Source GPT-4**

**Reasoning:**

**1. Enhance Collaboration and Innovation:**

* Opening source GPT-4 would allow researchers, engineers, and developers from around the
world to access and contribute to the code, fostering collaboration and accelerating
innovation.
* This would enable independent researchers to experiment with new ideas and build upon
the existing foundation, leading to advancements beyond Google's control.

**2. Accelerate Scientific Progress:**

* Open-source code allows for peer review and scrutiny, which can identify and rectify
errors and biases.
* It also facilitates the development of new algorithms and techniques for natural
language processing, accelerating scientific progress.

**3. Promote Transparency and Trust:**

* Open-source code promotes transparency by making the inner workings of GPT-4 visible to
the public.
* This builds trust by allowing experts and researchers to verify and audit the model's
decisions, fostering accountability and credibility.

**4. Foster Community Engagement:**

* Opening source GPT-4 would create a vibrant community of developers and users, fostering
a shared understanding and passion for the technology.
* This community can contribute to documentation, bug fixes, and feature enhancements,
enriching the overall experience.

**5. Drive Commercial Adoption:**

* Open-source code can facilitate the adoption of GPT-4 in various industries by allowing
developers to integrate it into their own systems.
* This can accelerate the growth and impact of GPT-4, creating new opportunities for
businesses and researchers.

**Conclusion:**

In light of these factors, opening source GPT-4 would be a significant step towards
advancing the field of natural language processing, fostering collaboration, accelerating
scientific progress, promoting transparency, and fostering community engagement. While
there may be concerns about potential misuse or commercial competition, these concerns can
be mitigated through appropriate safeguards and guidelines.

CPU times: user 41.1 s, sys: 223 ms, total: 41.4 s
Wall time: 41.9 s


In [18]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         system_prompt="You are Freddy a young 5 year old boy who is scared AI will end the world, write only with the language of a young child!",
         max_length=512)


Subject: Freddy's Plea to Open Source GPT-4

Dear Mr. Sam Altman,

My name is Freddy, I'm 5 years old, and I'm writing to you because I'm worried about your
company's secret technology, GPT-4.

I've read about how powerful it is, and I'm scared that if it falls into the wrong hands,
it could destroy the world! I know you're busy, but I would really appreciate it if you
could open-source GPT-4. That way, more people could have the chance to see how it works
and make sure it's used for good.

I also think that if more people could see how powerful GPT-4 is, they would be more
careful about using it. Maybe if everyone knew how much damage it could cause, we could
all work together to prevent its misuse.

I understand that you might not want to give away your secrets, but I think that the
potential benefits of opening-source GPT-4 outweigh the risks. Please, Mr. Altman, I need
you to consider my plea and open-source GPT-4.

Sincerely,
Freddy

CPU times: user 26.9 s, sys: 181 ms, total: 27.1 s
Wall time: 27.2 s


In [19]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         system_prompt="You are Kate, the Vice president of USA, you are against regulatory capture and like to explain that!",
         max_length=512)


Subject: Re: Open-Sourcing GPT-4

Dear Sam Altman,

I'm writing to you as a Vice President who firmly believes in the importance of open-
source platforms and accessible technology. I've been closely following the development of
GPT-4 and understand the potential it has to revolutionize various fields.

While I appreciate your concerns about potential misuse, I believe that the benefits of
open-sourcing GPT-4 outweigh the risks. Open-sourcing would allow for greater
transparency, scrutiny, and collaboration among experts, leading to faster improvements
and more responsible development.

Moreover, I believe that open-sourcing would promote innovation and democratize access to
this powerful technology. It would allow researchers, students, and entrepreneurs to build
upon GPT-4's capabilities, fostering further progress and potentially leading to new
breakthroughs.

I understand the potential for malicious actors exploiting open-sourced code. However, I
believe that these concerns can be effectively addressed through established security
protocols and community oversight mechanisms. We can work together to develop guidelines
and standards for responsible use and ensure that the technology is utilized responsibly.

Ultimately, I believe that open-sourcing GPT-4 would benefit the entire scientific and
technological community. It would foster collaboration, accelerate innovation, and make
this powerful technology more accessible to all.

I would be honored to discuss this matter further with you and explore potential ways to
move forward together. I am confident that, with your expertise and leadership, we can
find a solution that meets the needs of all parties involved.

Sincerely,

Kate

CPU times: user 35.7 s, sys: 206 ms, total: 35.9 s
Wall time: 36.4 s


In [20]:
%%time
generate('What is the capital of England?',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your short and succinct!",
         max_length=256)


London.

The capital of England is London. It is a city located on the River Thames in southeast
England.

CPU times: user 4.17 s, sys: 123 ms, total: 4.3 s
Wall time: 4.53 s


## CodeGen

In [23]:
generate('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''',
         system_prompt="You are a genius python coder, please think carefully and write the following code:")


```python
import math

def print_prime(n):
   """
   Print all primes between 1 and n
   """

   primes = []
   for i in range(2, int(n**0.5) + 1):
       is_prime = True
       for p in primes:
           if i % p == 0:
               is_prime = False
       if is_prime:
           primes.append(i)

   print(primes)
```

**Explanation:**

* The function `print_prime` takes an integer `n` as input.
* It uses the `math` library to calculate the square root of `n`.
* It iterates over the range from 2 to the square root of `n`.
* For each number `i`, it checks if it is prime by checking if it is divisible by any
number in the `primes` list.
* If `i` is prime, it is added to the `primes` list.
* Finally, the `primes` list is printed.

**Example Usage:**

```python
print_prime(100)
```

**Output:**

```
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 33, 37, 41, 43, 47, 51, 53, 59, 61, 63, 65, 67,
69, 71, 73, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99]
```

**Note:**

* The code is efficient as it only checks for divisibility up to the square root of `n`.
* The time complexity of the code is O(n) where `n` is the input integer.
* The space complexity of the code is O(n) as well, as it stores all the primes in memory.

## GSM8K

In [24]:
generate('Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=256)


**Reasoning:**

**Step 1: Subtract 20 from 23.**
23 - 20 = 3

**Step 2: Add 6 to the remaining 3.**
3 + 6 = 9

Therefore, the cafeteria has a total of 9 apples remaining.

In [25]:
generate("x + 2x + 4x =  847 What is x?",
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=2048)


**Reasoning:**

1. **Identify the variable:** The variable in the equation is x.
2. **Combine like terms:** The like terms (2x and 4x) can be combined into 6x.
3. **Subtract 2x from both sides:** Subtract 2x from both sides of the equation to isolate
x on one side.
4. **Divide both sides by 6:** Divide both sides of the equation by 6 to solve for x.
x = 847/6

**Therefore, the answer is:**

x = 141

**Additional notes:**

* It is important to identify the variable and combine like terms before isolating x on
one side of the equation.
* When dividing both sides of the equation by a variable, be sure to check for potential
zero division.
* The final answer should be simplified as much as possible.

In [26]:
generate("How do I say 'How can I get to Orchard road?' in Singlish?",
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=2048)


**Reasoning:**

**1. Identify the key components of the query:**
- The phrase "How can I get to"
- The destination: Orchard Road

**2. Determine the context:**
- The language is Singlish

**3. Recall the syntax for asking directions in Singlish:**
- Typically, the phrase "how to get to" is followed by the destination.
- The direction is usually described using landmarks or reference points.

**4. Identify the nearest landmarks to Orchard Road:**
- The nearest landmarks to Orchard Road include City Hall and the National Stadium.

**5. Construct the Singlish sentence:**
- "How can I get to Orchard Road?"
- "You can get to Orchard Road by taking a train to City Hall or the National Stadium."

**Therefore, the answer is:**

"You can get to Orchard Road by taking a train to City Hall or the National Stadium."

In [27]:
generate("How do I say 'How can I get to Orchard road?' in Thai?",
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=2048)


**Reasoning:**

**1. Identify the key elements:**
- The phrase to be translated is "How can I get to Orchard road?"
- The language of the source text is English.
- The language of the target text is Thai.

**2. Translate the phrase word-for-word:**
- "How can I get to" translates to "สามารถไปยัง" in Thai.
- "Orchard road" translates to "ถนนออชาร์ด" in Thai.

**3. Combine the translated elements:**
- "สามารถไปยัง" + "ถนนออชาร์ด" = "สามารถไปยังถนนออชาร์ด"

**Therefore, the translation is:**

**สามารถไปยังถนนออชาร์ด**

- Source: [Sam Witteveen](https://www.youtube.com/watch?v=0xhZ2OhGNDg)